## 使用 policy gradient

Policy gradient 是 RL 中另外一个大家族, 他不像 Value-based 方法 (Q learning, Sarsa), 但他也要接受环境信息 (observation), 不同的是他要输出不是 action 的 value, 而是具体的那一个 action, 这样 policy gradient 就跳过了 value 这个阶段. 而且个人认为 Policy gradient 最大的一个优势是: 输出的这个 action 可以是一个连续的值, 之前我们说到的 value-based 方法输出的都是不连续的值, 然后再选择值最大的 action. 而 policy gradient 可以在一个连续分布上选取 action

### Policy Gradient 之 Reinforce 算法
我们介绍的 policy gradient 的第一个算法是一种基于 整条回合数据 的更新, 也叫 REINFORCE 方法. 这种方法是 policy gradient 的最基本方法。

function Reinforce  
> initialize $\theta$ arbitrarily, where $\pi_\theta$ is the policy  
> for each episode $\{s_1,a_1,r_2,\ldots,s_{T-1},a_{T-1},r_T\} \sim \pi_\theta$ do  
>> for $t=1$ to $T-1$ do  
>>> $\theta \leftarrow \theta + \alpha \nabla_\theta \log \pi_\theta (s_t,a_t)v_t$  

>> end for

> end for
> return \theta

end function  

$\log(Policy(s,a))\cdot v$中的 $\log(Policy(s,a))$表示状态 $s$对所选动作 $a$ 的吃惊度，如果 $Policy(s,a)$ 概率越小，反向的 $\log(Policy(s,a))$ 越大。如果在$Policy(s,a)$很小的情况下，拿到了一个大的$R$，也就是$v$，那么$-\log(Policy(s,a))\cdot v$就更大，表示更吃惊，这就是$\log(Policy)\cdot v$的物理意义。

In [3]:
import gym
from RL_brain import PolicyGradient
import matplotlib.pyplot as plt

DISPLAY_REWARD_THRESHOLD = 400  # 在屏幕上显示模拟窗口会拖慢运行速度, 我们等计算机学得差不多了再显示模拟
RENDER = False  # 在屏幕上显示模拟窗口会拖慢运行速度, 我们等计算机学得差不多了再显示模拟

env = gym.make('CartPole-v0')
env.seed(1)     # 普通的 Policy gradient 方法, 使得回合的 variance 比较大, 所以我们选了一个好点的随机种子
env = env.unwrapped

print(env.action_space) # 显示可用 action
print(env.observation_space) # 显示可用 state 的 observation
print(env.observation_space.high) # 显示 observation 最高值
print(env.observation_space.low) # 显示 observation 最低值

# 定义
RL = PolicyGradient(
    n_actions=env.action_space.n,
    n_features=env.observation_space.shape[0],
    learning_rate=0.02,
    reward_decay=0.99, # gamma
    # output_graph=True, # 输出 tensorboard 文件
)

for i_episode in range(3000):
    """
    让计算机跑完一整个回合才更新一次。 之前的 Q leanring 等在回合中每一步都可以更新参数。
    """

    observation = env.reset()

    while True:
        if RENDER: env.render()

        action = RL.choose_action(observation)

        observation_, reward, done, info = env.step(action)

        RL.store_transition(observation, action, reward) # 存储这一回合的 transition

        if done:
            ep_rs_sum = sum(RL.ep_rs)

            if 'running_reward' not in globals():
                running_reward = ep_rs_sum
            else:
                running_reward = running_reward * 0.99 + ep_rs_sum * 0.01
            if running_reward > DISPLAY_REWARD_THRESHOLD: RENDER = True     # 判断是否显示渲染
            print("episode:", i_episode, "  reward:", int(running_reward))

            vt = RL.learn() # 学习, 输出 vt, 我们下节课讲这个 vt 的作用

            if i_episode == 0:
                plt.plot(vt)    # plot 这个回合的 vt
                plt.xlabel('episode steps')
                plt.ylabel('normalized state-action value')
                plt.show()
            
            observation = observation_

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Discrete(2)
Box(4,)
[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]


ValueError: Variable fc1/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/Users/jyjd/github/Tensorflow-LR/6_Policy_gradient_softmax/RL_brain.py", line 62, in _build_net
    activation=tf.nn.tanh,  # tanh activation
  File "/Users/jyjd/github/Tensorflow-LR/6_Policy_gradient_softmax/RL_brain.py", line 38, in __init__
    self._build_net()
  File "<ipython-input-2-5abb80d48334>", line 21, in <module>
    reward_decay=0.99,
